In [1]:
import os
import sys
import pandas as pd

In [2]:
%pwd

'c:\\Users\\justi\\Desktop\\MLOps Demo\\MLOPS-Demo-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\justi\\Desktop\\MLOps Demo\\MLOPS-Demo-Project'

In [5]:
sys.path.insert(0, '')


In [6]:
data = pd.read_csv("artifacts/data_ingestion/Heart_Disease (1).csv")
data = data.dropna(how='any',axis=0) 

data.isnull().sum()

sex                0
age                0
education          0
smokingStatus      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
CHDRisk            0
dtype: int64

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [8]:
from src.mlops_project.constants import *
from src.mlops_project.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    #may need to amend to add target Column
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        # target_col = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [10]:
import os
from src.mlops_project import logger

In [17]:
#Function may not be written well. Needs to exit immediately and return status if false, 
#Redundancy. Add Data Type validation
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            print(all_cols)
            all_schema = self.config.all_schema.keys()
            print(all_schema)
            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e



In [18]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-06-05 20:56:55,620: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-05 20:56:55,621: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-05 20:56:55,623: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-05 20:56:55,625: INFO: common: created directory at: artifacts]
[2024-06-05 20:56:55,625: INFO: common: created directory at: artifacts/data_validation]
['sex', 'age', 'education', 'smokingStatus', 'cigsPerDay', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose', 'CHDRisk']
dict_keys(['sex', 'age', 'education', 'smokingStatus', 'cigsPerDay', 'BPMeds', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose'])
